## Pandas DataFrame连接表，Merge, Join, Concat的对比
参考
- https://zhuanlan.zhihu.com/p/45442554

### 1.Join
- DataFrame的join和数据库中join 一样
- DataFrame的join默认为左连接
- DataFrame的join连接时，caller的关键列默认为index，可以使用参数on指定别的列为关键列，但是other的关键列永远都是index，所有使用别的列为关键列是，常常使用set_index()

docs: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html

In [1]:
import pandas as pd

In [2]:
caller = pd.DataFrame({'key':['K0', 'K1', 'K2', 'K3', 'K4', 'K5'], 'A':['A0', 'A1', 'A2', 'A3', 'A4', 'A5']})
caller.head()

,A,key
0,A0,K0
1,A1,K1
2,A2,K2
3,A3,K3
4,A4,K4


In [16]:
other = pd.DataFrame({'key':['K2', 'K1', 'K0','K99'], 'B':['B0', 'B1', 'B2', 'B99']})
other.head()

,B,key
0,B0,K2
1,B1,K1
2,B2,K0
3,B99,K99


In [19]:
# 通过索引连接，默认左连接
caller.join(other, lsuffix='_caller', rsuffix='_other')

,A,key_caller,B,key_other
0,A0,K0,B0,K2
1,A1,K1,B1,K1
2,A2,K2,B2,K0
3,A3,K3,B99,K99
4,A4,K4,NaN,NaN
5,A5,K5,NaN,NaN


In [28]:
# 通过列名连接
# 1.先把该列设置为索引，然后再连接，需要同时设定caller和other的索引，连接结果的索引也是指定的列
caller.set_index('key').join(other.set_index('key'))

,A,B
key,,
K0,A0,B2
K1,A1,B1
K2,A2,B0
K3,A3,NaN
K4,A4,NaN
K5,A5,NaN


In [29]:
# 2. 通过on参数指定连接的列
# DataFrame.join总是使用other的索引去连接caller，因此我们可以把指定的列设置为other的索引，
# 然后用on去指定caller的连接列，这样可以让连接结果的索引和caller一致
caller.join(other.set_index('key'), on='key')

,A,key,B
0,A0,K0,B2
1,A1,K1,B1
2,A2,K2,B0
3,A3,K3,NaN
4,A4,K4,NaN
5,A5,K5,NaN


### 2.merge

- 基于列索引合并，默认inner
- 默认以重叠列表作为连接键
- 可以多键连接，'on'参数后传入多键列表即可
- 如果两个对象的列表不同，可以用`left_on`, `right_on`指定。
- 也可以用行索引当连接键，使用参数`left_index=True`, `right_index=True`. 但是这种情况下最好用`join`

#### 与 join区别
- `join` 最适合的情况是基于行索引
- `merge` 基于列索引的合并

merge 与 join也能相互转换

In [35]:
pd.merge(caller, other)  # 默认 inner，默认以重叠列表作为连接键(这里是key)

,A,key,B
0,A0,K0,B2
1,A1,K1,B1
2,A2,K2,B0


### 3.concat
concat 轴向连接, 就是单纯地把两个表拼在一起，这个过程也被称作绑定（binding）或堆叠（stacking）。
这个函数的关键参数应该是 axis，用于指定连接的轴向。axis=1 在行中操作(水平堆叠) ，axis=0是在列中操作。默认是axis=0,即垂直堆叠。

In [45]:
# 按照行索引水平堆在了一起(水平堆叠)
pd.concat((caller, other), axis=1)
# 相对于
# pd.merge(caller, other, left_index=True, right_index=True, how='outer')
# 也相对于
# caller.join(other, lsuffix="l_")

,A,keyl_,B,key
0,A0,K0,B0,K2
1,A1,K1,B1,K1
2,A2,K2,B2,K0
3,A3,K3,B99,K99
4,A4,K4,NaN,NaN
5,A5,K5,NaN,NaN


In [51]:
# 垂直堆叠(axis=0, 默认)
pd.concat((caller, other), axis=0, ignore_index=True)  #  ignore_index为True表示把结果的合并表重新编排行索引， 否则用各自的index

/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,A,B,key
0,A0,NaN,K0
1,A1,NaN,K1
2,A2,NaN,K2
3,A3,NaN,K3
4,A4,NaN,K4
5,A5,NaN,K5
6,NaN,B0,K2
7,NaN,B1,K1
8,NaN,B2,K0
9,NaN,B99,K99
